# INSTALLING SELENIUM ON COLAB

In [ ]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin


     |████████████████████████████████| 911kB 5.1MB/s 
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease

In [ ]:
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')


# GETTING ALL AVAILABLE LINKS FROM CREDLY SITE


In [ ]:
from selenium import webdriver
# from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait  # for implicit and explict waits
from selenium.common.exceptions import NoSuchElementException 
from selenium.webdriver.common.keys import Keys
from time import sleep
import pandas as pd
import string
from tqdm import tqdm

def scrap_links(credit0,credit1,alpha=26):
  print("Getting all links, you have to wait...")
  temp_list = []
  print("\nChecking driver in catch or Downloading it ...")
  option = webdriver.ChromeOptions()
  option.add_argument('--headless')
  option.add_argument('--no-sandbox')
  option.add_argument('--disable-dev-shm-usage')
  driver = webdriver.Chrome('chromedriver',options=option)
  # driver = webdriver.Chrome(ChromeDriverManager().install(),options=option)

  driver.get("https://www.credly.com/users/sign_in")

  WebDriverWait(driver, timeout=3)
  username=driver.find_element_by_name ("email") 
  username.send_keys(credit0) 
  password =driver.find_element_by_name ("password") 
  password.send_keys(credit1) 
  login_button = driver.find_element_by_xpath ("//button[@type='submit']")    
  login_button.click()
  sleep(5)
  vm_button = driver.find_element_by_xpath ('//*[@id="website-header"]/div[4]/div/nav/div[1]') 
  vm_button.click()
  sleep(1)

  # print("fetching data...")
  for key in tqdm(string.ascii_lowercase[:alpha], position=0, leave=True):
      try:
          # print("for key ",key)
          searchbar=driver.find_element_by_id("search_input_2")
          searchbar.send_keys(Keys.BACKSPACE) 
          searchbar.send_keys(key) 
          sleep(3)
          vm_button = driver.find_element_by_xpath('/html/body/div[1]/div/div[2]/div/div/div[1]/nav/div[2]/ul/li[2]') 
          vm_button.click() 
          sleep(2)
          cfreq_source = driver.find_elements_by_xpath("/html/body/div[1]/div/div[2]/div/div/div[2]/div/div/div/div/a")
          for x in cfreq_source:
              # to check available attributes in element
              # attrs = driver.execute_script('var items = {}; for (index = 0; index < arguments[0].attributes.length; ++index) { items[arguments[0].attributes[index].name] = arguments[0].attributes[index].value }; return items;', x)
              # print(attrs)
              temp_list.append(x.get_attribute('href'))
              # print(x.get_attribute('href'))
          sleep(3)
      except NoSuchElementException as ex:
          print("Content not found!\nErr: ",ex)

  try:
    df = pd.DataFrame(list(set(temp_list)))
    df.to_csv('file_add.csv', index=False, header=False)
  except:
    print("Couldn't save file")
  print("quiting driver ...")
  driver.quit()
  return temp_list

# print(scrap_links('example@gmail.com','password',1))

# SCRAP PARAMETERED URLS

In [ ]:
from selenium import webdriver
# from webdriver_manager.chrome import ChromeDriverManager
from time import sleep
import pandas as pd
from tqdm import tqdm

def scrap_free(paratroops,add_list):
    print("geting para links")
    temp_list_free = []
    if len(add_list) > 0:
      temp_list = add_list
    else:
      try:
        df = pd.read_csv(r'file_add.csv')
        temp_list = df.iloc[:,0].tolist()
      except:
        print("Couldn't find file.")
        exit("Dead Process")
    print("\nChecking driver in catch or Downloading it ...")
    option = webdriver.ChromeOptions()
    option.add_argument('--headless')
    option.add_argument('--no-sandbox')
    option.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome('chromedriver',options=option)
    # driver.maximize_window()
    for i in tqdm(temp_list, position=0, leave=True):
        driver.get(i)
        try:
            cost_exist = driver.find_elements_by_xpath("/html/body/main/div[1]/div/div/div[2]/div[1]/ul/li/span[2]")
            for x in cost_exist:
                # to check available attributes in element
                # attrs = driver.execute_script('var items = {}; for (index = 0; index < arguments[0].attributes.length; ++index) { items[arguments[0].attributes[index].name] = arguments[0].attributes[index].value }; return items;', x)
                # print(attrs)
                if x.get_attribute("innerText") in paratroops:
                    print("\n",x.get_attribute("innerText")," link: ",i)
                    temp_list_free.append(i)
        except:
            print("Cost not found.")
    print("Closing browser")
    driver.close()
    try:
      if len(temp_list_free) > 0:
        df = pd.DataFrame(list(set(temp_list_free)))
        df.to_csv('para_list.csv', index=False, header=False)
    except:
      print("Couldn't save file")
    return temp_list_free
    
scrap_free(['Free'],scrap_links('example@gmail.com','password',26))
# other options are here:(choose any one)

# Validation
# Foundational
# Months
# Learning
# Intermediate
# Hours
# Experience
# Free
# Advanced


Getting all links, you have to wait...

Checking driver in catch or Downloading it ...


100%|██████████| 26/26 [03:41<00:00,  8.51s/it]


quiting driver ...
geting para links

Checking driver in catch or Downloading it ...


  1%|▏         | 16/1225 [00:08<10:24,  1.94it/s]


 Free  link:  https://www.credly.com/org/capacitacion-usach/badge/introduccion-a-kali


...

100%|█████████▉| 1224/1225 [10:30<00:00,  1.88it/s]


 Free  link:  https://www.credly.com/org/ibm/badge/ibm-z-federated-advocate-super-hero


100%|██████████| 1225/1225 [10:30<00:00,  1.94it/s]


Closing browser


['https://www.credly.com/org/capacitacion-usach/badge/introduccion-a-kali',
 'https://www.credly.com/org/east-noble-school-corporation/badge/science-a-z',
 'https://www.credly.com/org/fundacion-universitaria-los-libertadores/badge/introduccion-a-los-negocios-a-partir-de-simuladores',
 'https://www.credly.com/org/laticrete/badge/introduccion-a-autonivelantes',
 'https://www.credly.com/org/capacitacion-usach/badge/introduccion-a-linux',
 'https://www.credly.com/org/east-noble-school-corporation/badge/skype-a-scientist',
 'https://www.credly.com/org/el-puerto-de-liverpool/badge/atencion-a-comensales.2',
 'https://www.credly.com/org/el-puerto-de-liverpool/badge/atencion-a-comensales-2020',
 'https://www.credly.com/org/universidad-santo-tomas/badge/joven-investigador-a',
 'https://www.credly.com/org/pvh/badge/i-d-champion-level-1',
 'https://www.credly.com/org/ibm/badge/design-user-experience-d-ux-essentials',
 'https://www.credly.com/org/universidad-externado-de-colombia/badge/diseno-curri